# Main Script for LGBM Model

## Imports

In [2]:
# Imports
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import tree
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns
# Map
import importlib
mpl_toolkits = importlib.import_module('mpl_toolkits')
import mpl_toolkits
#from mpl_toolkits.basemap import Basemap
# Figures inline and set visualization style
%matplotlib inline
sns.set() #Different type of visualization

# Show multiple statements at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Gradient Boosting
import lightgbm as lgb

# Scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Imputer,LabelEncoder,PolynomialFeatures

## EDA

In [3]:
# Train Data
train = pd.read_csv('input/train.csv')

# Test Data
test = pd.read_csv('input/test.csv')

# Submission Data
sub = pd.read_csv('input/sample_submission.csv')

In [4]:
# We take a first look at the dataset
train.info()
print ('#################################################')
print ('#################################################')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658147 entries, 0 to 4658146
Data columns (total 6 columns):
id                          int64
molecule_name               object
atom_index_0                int64
atom_index_1                int64
type                        object
scalar_coupling_constant    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 213.2+ MB
#################################################
#################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2505542 entries, 0 to 2505541
Data columns (total 5 columns):
id               int64
molecule_name    object
atom_index_0     int64
atom_index_1     int64
type             object
dtypes: int64(3), object(2)
memory usage: 95.6+ MB


### Label Encoding

In [5]:
categoricals = train.select_dtypes(include='object').columns
categoricals = test.select_dtypes(include='object').columns

In [6]:
for c in categoricals:
    lbl = LabelEncoder()
    lbl.fit(list(train[c].values))
    train[c] = lbl.transform(list(train[c].values))

LabelEncoder()

LabelEncoder()

In [7]:
for c in categoricals:
    lbl = LabelEncoder()
    lbl.fit(list(test[c].values))
    test[c] = lbl.transform(list(test[c].values))

LabelEncoder()

LabelEncoder()

## Modeling

In [9]:
# We define de label
y = train['scalar_coupling_constant']

In [10]:
# We save the molecule names
molecules = train.pop('molecule_name')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658147 entries, 0 to 4658146
Data columns (total 5 columns):
id                          int64
atom_index_0                int64
atom_index_1                int64
type                        int64
scalar_coupling_constant    float64
dtypes: float64(1), int64(4)
memory usage: 177.7 MB


We drop the useless columns for training

In [13]:
X = train.drop(['id', 'scalar_coupling_constant'], axis=1)
X_test = test.drop(['id', 'molecule_name'], axis=1)

### Set K Folds

In [14]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)